### Serial Versus Parallel Processing

All of the computer programming that we have done this course to this point has been serial, that is, in series. In serial processing, each operation must wait for the previous operation to complete before it begins. In parallel processing, each cpu in a system can carry out a separate process simultaneously. This means that one operation that does not depend on another does not need to wait for the previous operation to complete.

#### The Sum of a Vector

![](../doc/img/reduce.png)

### MapReduce

MapReduce is a programming model designed to make parallel programming easier. The paradigm consists of two separate processes, a **mapper** and a **reducer**.

The mapper turns a document into a collection of key-value pairs. 

The reducer performs a specified operation over matching key-value pairs.

We have seen something similar before when working with Mongo aggregate functions. Consider the following Mongo aggregate function:

    {
        '$group': {
            '_id': '$word', 
            'count': { '$sum': 1 }
            }
    }

Here, we are grouping documents and then counting them by adding one to the count for each document. When we did this, we did not worry about the implementation. We simply told Mongo what we wanted for the output and let Mongo figure out how to get there. Using MapReduce we will begin to Add a bit more detail in terms of how we want a certain long-running process executed.

Let's think about how we might achieve the same end using the MapReduce paradigm. Note that we will be writing this in pure Python but typically this will be executed by a much larger and more powerful system such as Hive or Spark.

We can separate the above processed into two steps: 

1. the mapper that maps a document to a collection of word tokens
1. the reducer that reduces a collection of word tokens by adding one to the count associated with each word

### Test Data

The use the following list of strings as our "documents". This is to say that each string is a document that will be mapped.

In [1]:
documents = ["This paper presents a kernel-based principal component analysis, kernel PCA, to extract critical features for improving the performance of a stock trading model. ",
"The feature extraction method is one of the techniques to solve dimensionality reduction problems.",
"The kernel PCA is a feature extraction approach which has been applied to data transformation from known variables to capture critical information.",
"The kernel PCA is a kernel-based data mapping tool that has characteristics of both principal component analysis and non-linear mapping.",
"The feature selection method is another DRP technique that selects only a small set of features from known variables, but these features still indicate possible collinearity problems that fail to reflect clear information.",
"However, most feature extraction methods use a variable mapping application to eliminate noisy and collinear variables. In this research, we use the kernel-PCA method in a stock trading model to transform stock technical indices which allows features of smaller dimension to be formed.",
"The kernel-PCA method has been applied to various stocks and sliding window testing methods using both half-year and 1-year testing strategies. The experimental results show that the proposed method generates more profits than other DRP methods on the America stock market.",
"This stock trading model is very practical for real-world application, and it can be implemented in a real-time environment."]

Thus, one document looks like this

In [2]:
documents[0]

'This paper presents a kernel-based principal component analysis, kernel PCA, to extract critical features for improving the performance of a stock trading model. '

#### Word Tokens

Each token we generate will have the following form

    (word, 1)
    
In Python, we will represent this using a tuple consisting of a word string and the number 1, e.g.

    ('kernel-based', 1)

### Mapper

A complete mapper will return a list of properly formatted word tokens. Additionally, we need to have all punctuation removed.


In [3]:
def remove_punctuation(document):
    punctuation = ["'", '"', '`', '~', '!', '@', '#', '$', '%',
                   '^', '&', '*', '(', ')', '-', '_', '+', '=',
                   '{', '}', '[', ']', '|', '\\', ':', ';', ',',
                   '<', '>', '.', '/', '?']
    
    for symbol in punctuation:
        document = document.replace(symbol, '')
    return document

In [4]:
remove_punctuation(documents[0])

'This paper presents a kernelbased principal component analysis kernel PCA to extract critical features for improving the performance of a stock trading model '

When we built the mapper we use this `remove_punctuation` function and then create a list of words using the `.split()` method available to the `list` class. Note that the default argument for `.split()` is to split on whitespace. We have also used the `yield` argument to create a generator for lazy evaluation.

In [5]:
def mapper(document):
    document = remove_punctuation(document)
    words = document.split()
    for word in words:
        yield (word, 1)

In [6]:
mapper_0 = mapper(documents[0])

In [7]:
next(mapper_0)

('This', 1)

In [8]:
next(mapper_0)

('paper', 1)

In [9]:
next(mapper_0)

('presents', 1)

In [10]:
next(mapper_0)

('a', 1)

In practice we will pass these tokens to redis as they are created.

In [11]:
from redis import Redis 

redis_connection = Redis('this_redis')

We will use the `rpush` or "right push" to push each token to a Redis list using the word as the key.

In [12]:
def mapper(document, redis_connection):
    document = remove_punctuation(document)
    words = document.split()
    for word in words:
        token = 1
        redis_connection.rpush(word, token)
    
    return words

In [13]:
words = mapper(documents[0], redis_connection)

Later, we will use the `lpop` or "left pop" to pop each token from the front the left of the Redis list. Note that this is a FIFO or "first in, first out" pattern, that is, a queue.

In [14]:
words[6]

'component'

In [15]:
redis_connection.lpop('paper')

b'1'

In [16]:
words

['This',
 'paper',
 'presents',
 'a',
 'kernelbased',
 'principal',
 'component',
 'analysis',
 'kernel',
 'PCA',
 'to',
 'extract',
 'critical',
 'features',
 'for',
 'improving',
 'the',
 'performance',
 'of',
 'a',
 'stock',
 'trading',
 'model']

### Reducer

The reducer will operate in two steps:

1. It will pop each tokens for a given word into memory
2. It will reduce the list of tokens in Redis according to the operation we defined, in this case adding the second value from each token to the current count.

In [17]:
def reducer(word, redis_connection):
    count = 0
    token = redis_connection.lpop(word)
    while token:
        token = int(token.decode())
        count += token
        token = redis_connection.lpop(word)

    return count

In [18]:
words = mapper(documents[0], redis_connection)
words = mapper(documents[0], redis_connection)
words = mapper(documents[0], redis_connection)
words = mapper(documents[0], redis_connection)
words = mapper(documents[0], redis_connection)
reducer('component', redis_connection)

6

### Word List

The last thing to keep track of in this operation is a list of all the words seen. To do this we will use the `set` datatype also available in Redis via `.sadd()` and `.spop()`.

In [19]:
def mapper(document, redis_connection, word_list):
    document = remove_punctuation(document)
    words = document.split()
    for word in words:
        token = 1
        redis_connection.rpush(word, token)
        redis_connection.sadd(word_list, word)

### Word Count via MapReduce

In [20]:
def word_count(documents, redis_connection, word_list='word_list'):

    counts = []

    for document in documents:
        mapper(document, redis_connection, word_list)

    word = redis_connection.spop(word_list)
    while word:
        word = word.decode()
        count = reducer(word, redis_connection)
        counts.append((word, count))
        word = redis_connection.spop(word_list)
        
    return counts

In [21]:
documents_test = ["data science", "big data", "science fiction"]

In [22]:
word_count(documents_test, redis_connection)

[('science', 2), ('big', 1), ('fiction', 1), ('data', 2)]

In [23]:
word_count(documents, redis_connection)

[('component', 2),
 ('performance', 7),
 ('solve', 1),
 ('transform', 1),
 ('presents', 7),
 ('various', 1),
 ('In', 1),
 ('than', 1),
 ('halfyear', 1),
 ('using', 1),
 ('kernelPCA', 2),
 ('results', 1),
 ('extraction', 3),
 ('tool', 1),
 ('technical', 1),
 ('strategies', 1),
 ('variable', 1),
 ('kernelbased', 8),
 ('known', 2),
 ('realworld', 1),
 ('it', 1),
 ('still', 1),
 ('has', 3),
 ('reduction', 1),
 ('trading', 9),
 ('variables', 3),
 ('sliding', 1),
 ('principal', 8),
 ('kernel', 9),
 ('window', 1),
 ('other', 1),
 ('on', 1),
 ('stocks', 1),
 ('feature', 4),
 ('DRP', 2),
 ('testing', 2),
 ('been', 2),
 ('of', 11),
 ('profits', 1),
 ('problems', 2),
 ('and', 5),
 ('which', 2),
 ('environment', 1),
 ('in', 2),
 ('This', 8),
 ('market', 1),
 ('is', 5),
 ('techniques', 1),
 ('paper', 6),
 ('critical', 8),
 ('to', 15),
 ('nonlinear', 1),
 ('1year', 1),
 ('data', 2),
 ('the', 11),
 ('improving', 7),
 ('but', 1),
 ('a', 20),
 ('small', 1),
 ('realtime', 1),
 ('extract', 7),
 ('methods

##### Where can this operation be parallelized?